In [ ]:
# ===== Cell 1 =====
import os
import time as systime
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import simpson
from scipy.special import comb

from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2, Session

# ZNE folding (global)
from mitiq.zne.scaling import fold_global

# (for Aer simulation later)
from qiskit_aer import AerSimulator


In [ ]:
# ===== Cell 2 =====
# Batches, system dimension, frequency, partitions and shots
batches = 3
d = 4
w = 0.1
p = 30
nshots = 8192

q = 2 * int(np.ceil(np.log(d) / np.log(2)))
num_qubits = 2*q + 1


In [ ]:
# ===== Cell 3 =====
# Job dict (now includes scale factor as part of the key)
job_info_hadd4_qiskit_kingston_zne = {}

# Time grid
t_values = np.linspace(0, np.pi / w, p)


In [ ]:
# ===== Cell 4 =====
# Initialize the analytical number of non-null Walsh angles and the angles/indices arrays
num_angles = int(q**2/4 + q)
a_j = np.zeros(num_angles, dtype=int)
indices = np.zeros(num_angles, dtype=int)

# Pre-calculate constants for angle computations
result_h1 = -(8**(q/2))*(1+2**(q/2))/8
result_h2 = (4**q)/16

# Calculate angles and indices
for j1 in range(int(q/2)):
    base_index = j1 + (j1+1)*int(q/2)
    
    a_j[j1] = result_h1/(2**j1)
    a_j[base_index] = a_j[j1]
    
    indices[j1] = 2**j1
    indices[base_index] = 2**(j1 + int(q/2))

    for j2 in range(int(q/2)):
        offset_index = base_index + j2 + 1
        a_j[offset_index] = result_h2/(2**(j2+j1))
        indices[offset_index] = indices[base_index] + 2**j2


In [ ]:
# ===== Cell 5 =====
# Build the circuit segment for a given angle index (position), copy c (0 or 1), and time t
def gates(QC, position, c, t):
    binary_str = format(indices[position], f'0{q}b')[::-1]    
    msb_index = binary_str.rfind('1')

    # CNOTs before RZ
    for i in range(q):
        if binary_str[i] == '1' and i != msb_index:
            QC.cx(i + 1 + c*q, msb_index + 1 + c*q)
    
    # RZ
    theta_j = 2*a_j[position]*w*t/(2**q)
    QC.rz(theta_j, msb_index + 1 + c*q)
    
    # Undo CNOTs
    for i in reversed(range(q)):
        if binary_str[i] == '1' and i != msb_index:
            QC.cx(i + 1 + c*q, msb_index + 1 + c*q)


In [ ]:
# ===== Cell 6 (pure-Qiskit local folding, s=1.0 & 2.0) =====
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import ClassicalRegister
import numpy as np, os, time as systime

service = QiskitRuntimeService()
backend_name = "ibm_kingston"
backend = service.backend(backend_name)
sampler = SamplerV2(mode=backend)

# Optional while validating ZNE monotonicity:
# sampler.options.dynamical_decoupling.enable = False

job_info_path = "job_info_hadd4_qiskit_kingston_zne.npy"
job_info_hadd4_qiskit_kingston_zne = (
    np.load(job_info_path, allow_pickle=True).item()
    if os.path.exists(job_info_path) else {}
)

def build_unitary_circuit(t: float) -> QuantumCircuit:
    """Base circuit U with NO classical register or measurement."""
    qc = QuantumCircuit(num_qubits)
    for j in range(1, 2*q + 1):
        qc.h(j)
    for c in range(2):
        for position in range(num_angles):
            gates(qc, position, c, t)
    qc.h(0)
    for i in range(1, int(q / 2) + 1):
        qc.cswap(0, i, i + q)
    qc.h(0)
    return qc

def with_single_measure(circ: QuantumCircuit) -> QuantumCircuit:
    out = circ.copy()
    if out.num_clbits == 0:
        out.add_register(ClassicalRegister(1))
    out.measure(0, 0)
    return out

def fold_local_qiskit(U: QuantumCircuit, scale: float,
                      target_names=("cx","cswap","rz","h")) -> QuantumCircuit:
    """
    Local folding: choose ~half of target gates and replace G -> G G† G.
    Achieves s ≈ 1 + 2*k/N, where N=#target gates, k=#folded gates.
    For s=2.0, we set k ≈ N/2 deterministically.
    """
    if np.isclose(scale, 1.0):
        return with_single_measure(U)

    # Collect indices of foldable operations (exclude barriers / measurements)
    instrs = list(U.data)  # tuple of CircuitInstruction
    target_idx = [i for i, inst in enumerate(instrs)
                  if inst.operation.name in target_names]

    N = len(target_idx)
    if N == 0:
        # Nothing to fold: just measure
        return with_single_measure(U)

    # Desired number to fold for requested scale
    k = int(np.round((scale - 1.0) * N / 2.0))  # for s=2.0 => ~N/2
    k = max(0, min(k, N))

    # Deterministic, well-spread selection: take every other target
    # (if k != N//2 due to rounding, this keeps it close)
    pick = set(target_idx[1::2][:k])  # fold "odd" positions first

    # Rebuild circuit with folds in place
    V = QuantumCircuit(U.num_qubits)
    for i, inst in enumerate(instrs):
        op = inst.operation
        qargs = inst.qubits
        cargs = inst.clbits

        if i in pick:
            # Insert G, barrier, G†, barrier, G
            V.append(op, qargs, cargs)
            V.barrier()
            V.append(op.inverse(), qargs, cargs)
            V.barrier()
            V.append(op, qargs, cargs)
        else:
            V.append(op, qargs, cargs)

    # Add final measurement
    return with_single_measure(V)

# ------------------------------

scale_factors = [1.0, 2.0]

# Optional preview containers
results_preview = {}
job_ids_preview = {}

for t in t_values:
    results_preview[t] = {s: [] for s in scale_factors}
    job_ids_preview[t] = {s: [] for s in scale_factors}

    submitted = {(tt, bb, ss) for (tt, bb, ss) in job_info_hadd4_qiskit_kingston_zne.keys()
                 if np.isclose(tt, t)}

    print(f"\n🌀 Submitting jobs @ t = {t:.6f}")

    for batch in range(batches):
        U_base = build_unitary_circuit(t)

        for s in scale_factors:
            if (t, batch, s) in submitted:
                print(f"  ⚠️ Exists: t={t:.6f}, batch={batch}, s={s}, skipping.")
                continue

            qc_folded = fold_local_qiskit(U_base, s)

            # Keep optimization low to preserve folds
            transpiled = transpile(
                qc_folded, backend=backend,
                optimization_level=0, layout_method="sabre",
                routing_method="sabre", seed_transpiler=42
            )

            # (Optional) sanity check
            # print("scale", s, "depth:", transpiled.depth(), "ops:", transpiled.count_ops())

            batch_start = systime.time()
            while True:
                try:
                    job = sampler.run([transpiled], shots=nshots)
                    jid = job.job_id()
                    job_ids_preview[t][s].append(jid)
                    job_info_hadd4_qiskit_kingston_zne[(t, batch, s)] = jid
                    print(f"  ✅ Sent t={t:.6f}, batch={batch}, s={s}, job ID: {jid}")
                    break
                except Exception as e:
                    print(f"⚠️ Retry (t={t:.6f}, batch={batch}, s={s}): {e}")
                    systime.sleep(30)

            wait_start = systime.time()
            while job.status() not in ["DONE", "CANCELLED", "ERROR"]:
                elapsed = systime.time() - wait_start
                print(f"    ⏳ Status: {job.status()}, waiting... {elapsed:.1f}s ({elapsed/60:.2f} min)")
                systime.sleep(10)

            duration = systime.time() - batch_start
            with open("batch_times_log_zne.txt", "a") as f:
                f.write(f"t={t}, batch={batch}, s={s}, job_id={jid}, duration={duration:.2f}s ({duration/60:.2f}min)\n")

            if job.status() == "DONE":
                try:
                    result = job.result()
                    counts = result[0].join_data().get_counts()
                    results_preview[t][s].append(counts)
                    print(f"    ✅ Done. Counts (s={s}): {counts}")
                except Exception as e:
                    print(f"    ❌ Could not pull result (s={s}): {e}")
            else:
                print(f"    ❌ Job failed with status: {job.status()}")

# Persist job map
np.save(job_info_path, job_info_hadd4_qiskit_kingston_zne)
print("\n✅ All pending jobs handled & job_info saved.")


In [ ]:
# ===== Cell 7 (s1/s2 = SEM over all batches; ZNE = SEM over paired per-batch ZNE) =====
import numpy as np
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
job_info = np.load("job_info_hadd4_qiskit_kingston_zne.npy", allow_pickle=True).item()

t_all = sorted({t for (t, _, _) in job_info.keys()})

def mean_sem(vals):
    vals = np.asarray(vals, dtype=float)
    n = vals.size
    if n == 0:
        return np.nan, np.nan, 0
    if n == 1:
        return float(vals[0]), 0.0, 1
    sd = np.std(vals, ddof=1)
    return float(np.mean(vals)), float(sd / np.sqrt(n)), n

# Output containers (we only keep t’s where ZNE pairs exist so grids align)
t_list = []
s1_mean_list, s1_sem_list = [], []
s2_mean_list, s2_sem_list = [], []
zne_mean_list, zne_sem_list, zne_npairs_list = [], [], []

for t in t_all:
    # Collect ALL batches for s=1 and s=2 (for s1/s2 stats)
    s1_all, s2_all = [], []
    # Also collect by batch for pairing (for ZNE)
    s1_by_batch, s2_by_batch = {}, {}

    for (tt, batch, s), job_id in job_info.items():
        if not np.isclose(tt, t):
            continue
        s = float(s)
        if s not in (1.0, 2.0):
            continue
        try:
            job = service.job(job_id)
            if job.status() != "DONE":
                continue
            result = job.result()
            counts = result[0].join_data().get_counts()
            p0 = sum(v for k, v in counts.items() if k[::-1][0] == '0') / nshots
            Tr = 2 * p0 - 1
            if s == 1.0:
                s1_all.append(Tr)
                s1_by_batch[batch] = Tr
            else:
                s2_all.append(Tr)
                s2_by_batch[batch] = Tr
        except Exception as e:
            print(f"⚠️ Skipping (t={t}, batch={batch}, s={s}): {e}")

    # Build per-batch ZNE values for batches present in BOTH scales
    common_batches = sorted(set(s1_by_batch).intersection(s2_by_batch))
    if len(common_batches) == 0:
        continue  # no ZNE at this t -> skip to keep grids aligned

    zne_vals = np.array([2.0 * s1_by_batch[b] - s2_by_batch[b] for b in common_batches], dtype=float)

    # s1/s2: SEM over ALL batches available at that t (restores your original behavior)
    m1, sem1, _ = mean_sem(np.array(s1_all, dtype=float))
    m2, sem2, _ = mean_sem(np.array(s2_all, dtype=float))

    # ZNE: SEM across per-batch ZNE values (this is the fix)
    m0, sem0, n0 = mean_sem(zne_vals)

    t_list.append(t)
    s1_mean_list.append(m1); s1_sem_list.append(sem1)
    s2_mean_list.append(m2); s2_sem_list.append(sem2)
    zne_mean_list.append(m0); zne_sem_list.append(sem0); zne_npairs_list.append(n0)

# Convert to arrays
t_arr = np.array(t_list, dtype=float)
Tr_s1, err_s1 = np.array(s1_mean_list), np.array(s1_sem_list)   # SEM
Tr_s2, err_s2 = np.array(s2_mean_list), np.array(s2_sem_list)   # SEM
Tr_zne, err_zne = np.array(zne_mean_list), np.array(zne_sem_list)  # SEM (fixed)

# Save exactly what your plotting cell expects
np.savez("results_hadd4_qiskit_kingston_zne_s1.npz", t=t_arr, Tr=Tr_s1, err=err_s1)
np.savetxt("results_hadd4_qiskit_kingston_zne_s1.csv",
           np.column_stack((t_arr, Tr_s1, err_s1)),
           delimiter=",", header="t,Tr,err", comments="")

np.savez("results_hadd4_qiskit_kingston_zne_s2.npz", t=t_arr, Tr=Tr_s2, err=err_s2)
np.savetxt("results_hadd4_qiskit_kingston_zne_s2.csv",
           np.column_stack((t_arr, Tr_s2, err_s2)),
           delimiter=",", header="t,Tr,err", comments="")

np.savez("results_hadd4_qiskit_kingston_zne_extrap.npz", t=t_arr, Tr=Tr_zne, err=err_zne)
np.savetxt("results_hadd4_qiskit_kingston_zne_extrap.csv",
           np.column_stack((t_arr, Tr_zne, err_zne)),
           delimiter=",", header="t,Tr,err", comments="")

print(f"✅ Wrote s1/s2 SEM and ZNE SEM for {len(t_arr)} time points.")


In [ ]:
# ===== Cell 8 =====
# Divisors of integer n
def get_all_divisors(n):
    divisors = set()
    for i in range(1, int(n ** 0.5) + 1):
        if n % i == 0:
            divisors.add(i)
            divisors.add(n // i)
    return sorted(divisors)

# Analytical Fourier modes / reduced purity
def results(n):
    divisors = get_all_divisors(n)
    z = len(divisors) - 1
    R = sum(divisors)
    z_red = int(np.ceil(z/2))
    s = 0
    zt = z + 1
    for i in range(z_red, z + 1):
        if divisors[i] >= 2**(q/2):
            if divisors[i] != divisors[z - i]:
                s += divisors[i] + divisors[z - i]
                zt -= 2
            else:
                s += divisors[i]
                zt -= 1
    return R - s, zt

def calc_alpha_n_analyt(n):
    Rt, Zt = results(n)
    if n <= (2**(q/2)-1)*(2**(q/2)-1):
        return 4 / (4**q) * (Zt * (2**q + n) - (2**(q/2 + 1)) * Rt)
    else:
        return 0

def Trace_analytical(t):
    Tr_part = sum(calc_alpha_n_analyt(v) * np.cos(v * w * t) for v in range(1, int((2**(q/2) - 1)**2 + 1)))
    Tr_analit = ((2 * 8**(q/2) - 2**q) / 4**q) + Tr_part
    return Tr_analit


In [ ]:
# ===== Final Cell (no Aer; matched colors & captions across both plots) =====
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from scipy.integrate import simpson

# --- Load processed traces (s=1.0 & s=2.0 + ZNE extrapolated) ---
t_s1, Tr_s1, err_s1 = np.loadtxt("results_hadd4_qiskit_kingston_zne_s1.csv",
                                 delimiter=",", skiprows=1, unpack=True)
t_s2, Tr_s2, err_s2 = np.loadtxt("results_hadd4_qiskit_kingston_zne_s2.csv",
                                 delimiter=",", skiprows=1, unpack=True)
t_zne, Tr_zne, err_zne = np.loadtxt("results_hadd4_qiskit_kingston_zne_extrap.csv",
                                    delimiter=",", skiprows=1, unpack=True)

# Ensure aligned grids
assert np.allclose(t_s1, t_s2) and np.allclose(t_s1, t_zne), "t-grids for s1/s2/ZNE differ."
t_val = t_zne

# --- Color palette (shared by both plots) ---
COLORS = {
    "analytical":  "red",         # match your method
    "zne_s1":      "blue",        # ZNE s=1.0 (match 'Demonstration (Original)')
    "zne_s2":      "#e377c2",     # ZNE s=2.0 (magenta; distinct from others)
    "zne_extrap":  "darkorange",  # ZNE extrapolated (match 'Demonstration (Mitigated)')
    "Bn":          "green",       # lower bound
    "Pn":          "purple",      # composite bound
    "region":      "cornflowerblue",
}

# --- Analytical trace on the same t-grid ---
Tr_val_analyt = np.array([Trace_analytical(t) for t in t_val])

# --- Helper for Fourier modes from traces ---
def calc_alpha_n_from_trace(n, tvals, Tr_vals, Tr_errs=None):
    integrando = (2*w/np.pi) * Tr_vals * np.cos(n * w * tvals)
    val = simpson(integrando, x=tvals)
    if Tr_errs is None:
        return val, 0.0
    errors_integrando = (2*w/np.pi) * Tr_errs * np.abs(np.cos(n * w * tvals))
    delta_t = np.pi/(w*(len(tvals)-1))
    err = np.sqrt(np.sum((delta_t * errors_integrando)**2))
    return val, err

# --- Bounds / analytics helpers (same formulas & colors as your method) ---
def analytical_bound(n):
    n = np.asarray(n)
    cond = (n >= 2) & (n < 2**(q/2))
    result = np.zeros_like(n, dtype=float)
    result[cond] = (8 / 2**(2*q)) * (-(2**(q/2) - 1)*n[cond] + 2**(q/2)*(2**(q/2) - 1))
    return result

def alpha_p2_func(x):
    base = (4/(2**(q/2))**4)*((2**(q/2))**2 + x - 2 * (2**(q/2)) * np.sqrt(x))
    return np.where(x < d, base + analytical_bound(x), base)

# --- n-grid & analytics ---
n_val = np.arange(2, 2*(d-1)+1)
alpha_n_analyt = np.array([calc_alpha_n_analyt(n) for n in n_val])
bound_n = analytical_bound(n_val)
alpha_p2 = alpha_p2_func(n_val)

# --- Fourier modes for ZNE datasets ---
alpha_n_s1, alpha_n_err_s1 = map(np.array, zip(*[
    calc_alpha_n_from_trace(n, t_val, Tr_s1, err_s1) for n in n_val
]))
alpha_n_s2, alpha_n_err_s2 = map(np.array, zip(*[
    calc_alpha_n_from_trace(n, t_val, Tr_s2, err_s2) for n in n_val
]))
alpha_n_zne, alpha_n_err_zne = map(np.array, zip(*[
    calc_alpha_n_from_trace(n, t_val, Tr_zne, err_zne) for n in n_val
]))

# =========================
# Plot 1: Purity vs time
# =========================
plt.figure()

# Analytical (solid red line)
plt.plot(t_val, Tr_val_analyt, color=COLORS["analytical"], label='Analytical', linewidth=2)

# ZNE raw scales (lines guiding points)
plt.errorbar(t_val, Tr_s1, yerr=err_s1, fmt='-o', capsize=5, markersize=5,
             color=COLORS["zne_s1"], ecolor=COLORS["zne_s1"], label=f'ZNE (ξ = 1.0)')
plt.errorbar(t_val, Tr_s2, yerr=err_s2, fmt='-s', capsize=5, markersize=5,
             color=COLORS["zne_s2"], ecolor=COLORS["zne_s2"], label=f'ZNE (ξ = 2.0)')

# ZNE extrapolated (line + diamond markers, darkorange)
plt.errorbar(t_val, Tr_zne, yerr=err_zne, fmt='-D', capsize=5, markersize=6,
             color=COLORS["zne_extrap"], ecolor=COLORS["zne_extrap"], label='ZNE (Extrapolated)')


plt.xlabel('t', fontsize=16)
plt.ylabel(r'$\gamma_A(t)$', fontsize=16)
plt.title(f'Purity vs time for $d = {d}$ (ZNE)', fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True)

# Legend order: Analytical, ZNE s=1.0, ZNE s=2.0, ZNE (Extrapolated)
handles, labels = plt.gca().get_legend_handles_labels()
desired = ['Analytical', 'ZNE (ξ = 1.0)', 'ZNE (ξ = 2.0)', 'ZNE (Extrapolated)']
order = [labels.index(x) for x in desired if x in labels]
plt.legend([handles[i] for i in order], [labels[i] for i in order], fontsize=9.5)
plt.tight_layout()
plt.savefig("purity_vs_time_zne_d4.pdf", dpi=300, bbox_inches='tight')
plt.show()

# =========================
# Plot 2: Fourier modes
# =========================
plt.figure()

# Analytical (solid red line + markers)
plt.plot(n_val, alpha_n_analyt, marker='o', linestyle='-',
         color=COLORS["analytical"], label='Analytical')

# ZNE raw scales
plt.errorbar(n_val, alpha_n_s1, yerr=alpha_n_err_s1, fmt='-o',
             capsize=5, color=COLORS["zne_s1"], ecolor=COLORS["zne_s1"],
             label='ZNE (ξ = 1.0)')
plt.errorbar(n_val, alpha_n_s2, yerr=alpha_n_err_s2, fmt='-s',
             capsize=5, color=COLORS["zne_s2"], ecolor=COLORS["zne_s2"],
             label='ZNE (ξ = 2.0)')

# ZNE extrapolated
plt.errorbar(n_val, alpha_n_zne, yerr=alpha_n_err_zne, fmt='-D',
             capsize=5, color=COLORS["zne_extrap"], ecolor=COLORS["zne_extrap"],
             label='ZNE (Extrapolated)')

# Bounds / region (match your colors)
plt.plot(n_val, bound_n, color=COLORS["Bn"], label=r'Lower bound $B_n$')
plt.plot(n_val, alpha_p2, color=COLORS["Pn"], linewidth=2.2, label=r'Composite bound $P_n$')

# Fill region & include in legend via proxy patch
mask = (n_val >= 2) & (n_val <= int((d - 1)**2))
y_alpha_p2_full = alpha_p2[mask]
x_full = n_val[mask]
# replace your fill_between + proxy with this single call
plt.fill_between(x_full, bound_n[mask], y_alpha_p2_full,
                 color=COLORS["region"], alpha=0.7, label='Possibly prime')

# then build the legend in your chosen order
handles, labels = plt.gca().get_legend_handles_labels()
desired = [
    'Analytical',
    'ZNE (s = 1.0)',
    'ZNE (s = 2.0)',
    'ZNE (Extrapolated)',
    r'Lower bound $B_n$',
    r'Composite bound $P_n$',
    'Possibly prime'
]
order = [labels.index(x) for x in desired if x in labels]
plt.legend([handles[i] for i in order], [labels[i] for i in order])


plt.xlabel('n', fontsize=16)
plt.ylabel(r'$\alpha_n$', fontsize=16)
plt.title(f'Fourier modes for $d = {d}$ (ZNE)', fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.legend(handles=handles, labels=labels, fontsize=9.5)
plt.grid(True)
plt.tight_layout()
plt.savefig("fourier_modes_zne_d4.pdf", dpi=300, bbox_inches='tight')
plt.show()